In [14]:
import pandas as pd
import os
import re
import numpy as np
from collections import Counter
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score


Per creare gli embeddings non ci serve dividere il dataset in test e train ma è utile trainare e quindi selezionare le features della matrice in base alle frequenze incontrate  nel corpus. Questo implica che un corpus più ampio ci aiuta ad una migliore selezione considerando che non dovremmo avere limiti a livello computazionale.

In [2]:
#Data loading
data_1 = pd.read_csv("Dati/train.csv")
data_2 = pd.read_csv("Dati/test.csv")
data = pd.concat([data_1, data_2], axis=0, ignore_index=True)


Nella fase di preprocessing si cerca di diminuire la varianza di termini che può essere creata da typo o comunque contenuto non informativo. Data la natura sparsa del tipo di rappresentazione del testo scelta (TF-IDF) questo aiuta la selezione delle features. 

In [3]:
def text_cleaner(text):
    '''
    la funzione prende una stringa e la pulisce secondo i criteri scritti nei commenti
        input= text
        otput= text
    '''
    #l'obbiettivo è diminuire il numero di termini
    text = text.lower()
    text = re.sub(r'\d+', ' ', text)#eliminare i numeri perchè non hanno informazioni sul sentiment
    text = re.sub(r'<.*?>', ' ', text)#elimina il testo utilizzato per html e.g <br \>
    text = re.sub(r'(--+)', ' ', text)  #elimina - quando sono ripetuti
    text = re.sub(r"[;%#'()_:=+,.*|~^<>@{}/?\"\[\]\[\\]"," ",text) # rimuove la punteggiatura e i simboli tranne il punto esclamativo
    return text

In [4]:
#creazione delle colonne pulite
data['clean_review'] = data['review'].apply(text_cleaner)

In [5]:

#review 
vectorizer = CountVectorizer()
count = vectorizer.fit_transform(data['review'].tolist())
vocabolario = dict(zip(vectorizer.get_feature_names_out().tolist(),count.sum(axis=0).A1.tolist()))
vocabolario_ordinato = dict(sorted(vocabolario.items(), key=lambda x: x[1], reverse=True))
#review pulite
vectorizer_2= CountVectorizer()
count_clean = vectorizer_2.fit_transform(data['clean_review'].tolist())
vocabolario_clean = dict(zip(vectorizer_2.get_feature_names_out().tolist(),count_clean.sum(axis=0).A1.tolist()))
vocabolario_ordinato_clean = dict(sorted(vocabolario_clean.items(), key=lambda x: x[1], reverse=True))

keys_vocabolario = set(vocabolario_ordinato.keys())
keys_vocabolario_clean = set(vocabolario_ordinato_clean.keys())

keys_in_common = keys_vocabolario.intersection(keys_vocabolario_clean)

# Trova le parole presenti solo prima del prerpocessing
keys_only_before = keys_vocabolario - keys_vocabolario_clean

# Trova le parole presenti solo dopo il preprocessing
keys_only_after = keys_vocabolario_clean - keys_vocabolario

print(f"Prima del preprocessing possimao individuare {len(keys_only_before)} parole che non sono presenti dopo il preprocessing, mentre dopo il preprocessing abbiamo {len(keys_only_after)} parole che non sono presenti prima del preprocessing")

conteggio_valori = Counter([vocabolario_ordinato_clean[i] for i in keys_only_after])
conteggio_valori = dict(sorted(conteggio_valori.items()))
print(conteggio_valori)

Prima del preprocessing possimao individuare 3555 parole che non sono presenti dopo il preprocessing, mentre dopo il preprocessing abbiamo 212 parole che non sono presenti prima del preprocessing
{1: 185, 2: 20, 5: 1, 6: 2, 7: 1, 8: 2, 18: 1}


In [10]:
tfidf_vectorizer = TfidfVectorizer(
#scelgo di pendere i bi-grams perchè potrebbero esserci delle negazioni che sono informative    
                                   ngram_range=(1,2), 
                                   stop_words='english', 
                                   analyzer='word',
#metto questo filtro perchè ci sono typo e parole inventate tipo usernames che andrebbero solo ad appesantire  
                                   min_df=0.001, 
                                   max_df=0.7,
                                   sublinear_tf=True, 
                                   use_idf=True)
#creiamo la matrice dei tfidf in base a tutti i documenti del corpus in modo da avere un numero di colonne che rappresentano
#quell che in base al corpus potrebbero essere delle parole significative per la comprensione del label
tfidf_vectorizer.fit(data['clean_review'])
# eliminiamo le review che non hanno il label considerando che vogliamo fare dell'aprendimento supervisionato
data_filtered = data.dropna(subset=['label']).copy()
#generiamo la matrice in base alle fetures precedentemente create su tutto il corpus e ai documenti con cui vogliamo fare la classificazione
data_filtered = data.groupby('label').apply(lambda x: x.sample(2000, replace=False)).reset_index(drop=True)
X = tfidf_vectorizer.transform(data_filtered['clean_review'])


In [11]:
# associamo il label ad un valore numerico
le = LabelEncoder()
data_filtered['label_encoded'] = le.fit_transform(data_filtered['label'])
#classificazione
y = data_filtered['label_encoded']
#kernel choice 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
#X_cv, X_val, y_cv, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=77, stratify=y)

modello = svm.SVC(kernel='poly')

# Definizione della griglia degli iperparametri da esplorare
parametri = {
    'degree': [1, 2, 3, 4, 5],  # Varia il grado del polinomio
    'C': [0.1, 1, 10]      # Varia il parametro di costo C
}

# Creazione dell'oggetto GridSearchCV
grid_search = GridSearchCV(modello, parametri, cv=5, scoring='accuracy')

# Esecuzione della Grid Search sulla versione standardizzata dei dati di addestramento
grid_search.fit(X_train, y_train)
print("Iperparametri ottimali:", grid_search.best_params_)

Iperparametri ottimali: {'C': 1, 'degree': 1}


In [15]:

linear_svm =  svm.SVC(kernel='poly', degree=grid_search.best_params_['degree'], C=grid_search.best_params_['C'] )
linear_svm.fit(X_train, y_train)
y_pred = linear_svm.predict(X_test)
classification_report(y_test, y_pred)
# Stampa di precisione, richiamo e F1 per ciascuna classe
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(report)


Accuracy: 0.85125
              precision    recall  f1-score   support

           0       0.85      0.85      0.85       400
           1       0.85      0.85      0.85       400

    accuracy                           0.85       800
   macro avg       0.85      0.85      0.85       800
weighted avg       0.85      0.85      0.85       800



In [16]:

# eliminiamo le review che non hanno il label considerando che vogliamo fare dell'aprendimento supervisionato
data_filtered = data.dropna(subset=['label']).copy()
#generiamo la matrice in base alle fetures precedentemente create su tutto il corpus e ai documenti con cui vogliamo fare la classificazione
X = tfidf_vectorizer.transform(data_filtered['clean_review'])
# associamo il label ad un valore numerico
le = LabelEncoder()
data_filtered['label_encoded'] = le.fit_transform(data_filtered['label'])
#classificazione
y = data_filtered['label_encoded']
#kernel choice 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

linear_svm =  svm.SVC(kernel='linear')
linear_svm.fit(X_train, y_train)
y_pred = linear_svm.predict(X_test)
# Stampa di precisione, richiamo e F1 per ciascuna classe
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(report)

Accuracy: 0.8936
              precision    recall  f1-score   support

           0       0.90      0.88      0.89      5000
           1       0.88      0.90      0.89      5000

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

